# Loan Approval Prediction (Classification)

Dataset Link: https://www.kaggle.com/datasets/architsharma01/loan-approval-prediction-dataset

Algorithms: Logistic Regression, Random Forest


## Load Dataset

In this section, we will load the dataset from the provided link and perform initial data exploration.